In [38]:
import os
import json
import pickle
import joblib
import pandas as pd
from flask import Flask, jsonify, request
from peewee import (
    Model, IntegerField, FloatField,
    TextField, IntegrityError,BooleanField
)
from playhouse.shortcuts import model_to_dict
from playhouse.db_url import connect

import requests
from numpy import nan
import numpy as np

In [41]:
df = pd.read_csv('data/train.csv')
df.head(1)

,observation_id,Type,Date,Part of a policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Legislation,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing,station
0,34d76816-cfc2-4bdd-b3a2-bf0c40b12689,Person search,2019-12-01T00:00:00+00:00,True,NaN,NaN,Male,18-24,Asian/Asian British - Any other Asian background,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,NaN,False,devon-and-cornwall


In [42]:
columns = ['observation_id', 'Type', 'Date', 'Part of a policing operation',
       'Latitude', 'Longitude', 'Gender', 'Age range',
       'Officer-defined ethnicity', 'Legislation',
       'Object of search', 'station']

random_row = df[columns].loc[8]

In [43]:
lista = []
for i in columns:
    if isinstance(random_row[i], np.float64):
        print("'"+i+"'",":",random_row[i], ",")
    elif isinstance(random_row[i], bool):
        print("'"+i+"'",":",random_row[i], ",")
    else:
        print("'"+i+"'",":","'"+random_row[i]+"'", ",")


'observation_id' : '2a5d011a-6ff1-4582-ae91-20b6990ffbc9' ,
'Type' : 'Person search' ,
'Date' : '2019-12-01T00:25:00+00:00' ,
'Part of a policing operation' : True ,
'Latitude' : nan ,
'Longitude' : nan ,
'Gender' : 'Male' ,
'Age range' : '18-24' ,
'Officer-defined ethnicity' : 'Black' ,
'Legislation' : 'Misuse of Drugs Act 1971 (section 23)' ,
'Object of search' : 'Controlled drugs' ,
'station' : 'devon-and-cornwall' ,


In [52]:
observation_request  = {'observation_id' : '3' ,
'Type' : 'Person search' ,
'Date' : '2019-12-01T00:25:00+00:00' ,
'Part of a policing operation' : True ,
'Latitude' : nan ,
'Longitude' : nan ,
'Gender' : 'Male' ,
'Age range' : '18-24' ,
'Officer-defined ethnicity' : 'Black' ,
'Legislation' : 'Misuse of Drugs Act 1971 (section 23)' ,
'Object of search' : 'Controlled drugs' ,
'station' : 'devon-and-cornwall'}

In [45]:
observation_request.keys()

dict_keys(['observation_id', 'Type', 'Date', 'Part of a policing operation', 'Latitude', 'Longitude', 'Gender', 'Age range', 'Officer-defined ethnicity', 'Legislation', 'Object of search', 'station'])

In [46]:
def check_valid_column(observation):
    """
        Validates that our observation only has valid columns
        
        Returns:
        - assertion value: True if all provided columns are valid, False otherwise
        - error message: empty if all provided columns are valid, False otherwise
    """
    
    valid_columns = {
                    "observation_id",
                    "Type",
                    "Date",
                    "Part of a policing operation",
                    "Latitude",
                    "Longitude",
                    "Gender",
                    "Age range",
                    "Officer-defined ethnicity",
                    "Legislation",
                    "Object of search",
                    "station"
}

    # print(observation)
    keys = set(observation.keys())
    
    if len(valid_columns - keys) > 0: 
        missing = valid_columns - keys
        error = "Missing columns: {}".format(missing)
        return False, error
    
    if len(keys - valid_columns) > 0: 
        extra = keys - valid_columns
        error = "Unrecognized columns provided: {}".format(extra)
        return False, error    

    return True, ""

In [47]:
observation_request

{'observation_id': '3',
 'Type': 'Person search',
 'Date': '2019-12-01T00:25:00+00:00',
 'Part of a policing operation': True,
 'Latitude': nan,
 'Longitude': nan,
 'Gender': 'Male',
 'Age range': '18-24',
 'Officer-defined ethnicity': 'Black',
 'Legislation': 'Misuse of Drugs Act 1971 (section 23)',
 'Object of search': 'Controlled drugs',
 'station': 'devon-and-cornwall'}

In [49]:
# obs_dict = observation_request
# bn=obs_dict.copy()

# columns_ok, error = check_valid_column(observation)
# if not columns_ok:
#     response = {'error': error}

# _id = obs_dict['observation_id']
# observation.pop('observation_id')



In [34]:
observation

{'Type': 'Person search',
 'Date': '2019-12-01T00:25:00+00:00',
 'Part of a policing operation': True,
 'Latitude': nan,
 'Longitude': nan,
 'Gender': 'Male',
 'Age range': '18-24',
 'Officer-defined ethnicity': 'Black',
 'Legislation': 'Misuse of Drugs Act 1971 (section 23)',
 'Object of search': 'Controlled drugs',
 'station': 'devon-and-cornwall'}

In [35]:
check_valid_column(observation_request)

(True, '')

In [3]:
url = 'http://127.0.0.1:5000/should_search'
r = requests.post(url)
r.ok

response = r.json()
response

NameError: name 'requests' is not defined

In [64]:
url = 'http://127.0.0.1:5000/list-db-contents'

r = requests.post(url, json=observation_request)
r.ok

response = r.json()
response

JSONDecodeError: Expecting value: line 1 column 1 (char 0)